<a href="https://colab.research.google.com/github/ssvadla/Sharing/blob/main/Co_training_3C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def training(unlabel_1,train,i,j):
    train_size = len(train)
    train_size_list.append(train_size)
    i_list.append(i)
    j_list.append(j)
    print("length of the UL received",len(unlabel_1))
    
   
    import pandas as pd
    from sklearn.model_selection import KFold
    from sklearn.model_selection import cross_val_score

    from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

#     train1 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data1.csv')
#     train2 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data2.csv')
#     train3 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data3.csv')
#     train4 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data4.csv')
#     train5 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data5.csv')
#     train6 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data6.csv')
#     train7 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data7.csv')
#     train8 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data8.csv')
#     train9 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data9.csv')
#     train10 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data10.csv')
#     train_highKappa = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data_highkappa.csv')

#     train1.head()
#     train = train1
#     train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
#     for i in train_list:
#         #print(i)
#         train = train.append(i)

#     #unlabel_size = unlabel_size
#     #Threshold=Threshold
#     train.sort_values("Sentence", inplace = True)
#     print(len(train))

#     train = train.drop_duplicates(subset ="Sentence")

#     train['Target'].unique()


#     train['Target']=train['Target'].replace(['Others'],'Invalid')
#     train['Target'].unique()
#     print(len(train))



    #cleaning
    import nltk
    import re
    import string
    nltk.download('stopwords')
    nltk.download('wordnet')
    stopword=nltk.corpus.stopwords.words('english')
    from nltk.stem import WordNetLemmatizer
    wl= WordNetLemmatizer()

    def clean_text(text):
      text="".join([word.lower() for word in text if word not in string.punctuation])
      tokens = re.split('\W+',text)
      text = [wl.lemmatize(word) for word in tokens if word not in stopword]
      return text

    from sklearn.feature_extraction.text import TfidfVectorizer

    tfidf_vect = TfidfVectorizer(analyzer = clean_text)
    #X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
    X_tfidf = tfidf_vect.fit_transform(train['text'])
    print(X_tfidf.shape)

    test = pd.read_csv(r'C:\Users\iia\Documents\Supriya\test_data.csv')

    test['Target']=test['Target'].replace(['Others'],'Invalid')
    test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
    test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
    from nltk.corpus import stopwords
    words = stopwords.words('english')
    test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
    t_p = tfidf_vect.transform(test['Sentence'])

    import lightgbm as lgb
    from sklearn import svm
    from sklearn.ensemble import RandomForestClassifier

    classifier_rf = RandomForestClassifier(n_estimators = 350, criterion = 'gini', max_features = 'auto', random_state = 42)
    classifier_lgb = lgb.LGBMClassifier()
    classifier_svm = svm.LinearSVC(multi_class='ovr',class_weight='balanced')

    #train = train.rename(columns={'Sentence':'text'})
    train.head()


    import numpy as np
    from sklearn.datasets import make_classification

    from sklearn.pipeline import make_pipeline
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import confusion_matrix
    from sklearn.base import BaseEstimator, ClassifierMixin
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import average_precision_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import cohen_kappa_score
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report



    #X, y = make_classification(random_state=1)

    X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.20,random_state=42)

    X_train_whole = X_tfidf
    Y_train_whole = train['Target']
    #X_train_whole, x_val_whole, Y_train_whole, y_val_whole = train_test_split(X_tfidf,train['Target'],test_size=0.1,random_state=42)

    
    classifier_rf.fit(X_train, Y_train)
    classifier_lgb.fit(X_train, Y_train)
    classifier_svm.fit(X_train, Y_train)

    val_pred_rf = classifier_rf.predict(x_val)
    val_pred_lgb = classifier_lgb.predict(x_val)
    val_pred_svm = classifier_svm.predict(x_val)

    Accuracy_score_rf = accuracy_score(y_val,val_pred_rf)
    print("Accuracy_score_rf ",Accuracy_score_rf)
    Accuracy_score_lgb = accuracy_score(y_val,val_pred_lgb)
    print("Accuracy_score_lgb ",Accuracy_score_lgb)
    Accuracy_score_svm = accuracy_score(y_val,val_pred_svm)
    print("Accuracy_score_svm ",Accuracy_score_svm)


    classification_report_test_rf = classification_report(y_val,val_pred_rf)
    print("classification report rf",classification_report_test_rf)
    classification_report_test_lgb = classification_report(y_val,val_pred_lgb)
    print("classification report lgb",classification_report_test_lgb)
    classification_report_test_svm = classification_report(y_val,val_pred_svm)
    print("classification report svm ",classification_report_test_svm)
    
    cv = KFold(n_splits=5, random_state=1, shuffle=True)
    cross_valid_rf = cross_val_score(classifier_rf,x_val ,y_val, scoring = 'f1_weighted', cv=cv, n_jobs=-1).mean()
    print("cross validation score RF",cross_valid_rf)
    cross_valid_lgb = cross_val_score(classifier_lgb,x_val ,y_val, scoring = 'f1_weighted', cv=cv, n_jobs=-1).mean()
    print("cross validation score LGB",cross_valid_lgb)
    cross_valid_svm = cross_val_score(classifier_svm,x_val ,y_val, scoring = 'f1_weighted', cv=cv, n_jobs=-1).mean()
    print("cross validation score SVM",cross_valid_svm)


    f1_score_rf = f1_score(y_val,val_pred_rf,average='weighted')
    print(" f1 score test rf ",f1_score_rf )
    f1_score_lgb = f1_score(y_val,val_pred_lgb, average='weighted')
    print(" f1 score test lgb ",f1_score_lgb )
    f1_score_svm = f1_score(y_val,val_pred_svm, average='weighted')
    print(" f1 score test svm ",f1_score_svm )

    if (Accuracy_score_rf > Accuracy_score_lgb) and (Accuracy_score_rf > Accuracy_score_svm):
      B = classifier_rf
    elif (Accuracy_score_svm > Accuracy_score_lgb) and (Accuracy_score_svm > Accuracy_score_rf):
      B = classifier_svm
    else:
      B = classifier_lgb


    B.fit(X_train_whole,Y_train_whole)
    test_pred = B.predict(t_p)
    accuracy_test = accuracy_score(test['Target'],test_pred)
    print('Accuracy test data', accuracy_test)
    f1_score_test = f1_score(test['Target'],test_pred, average='weighted')
    print(" f1 score test ",f1_score_test )
    
    f1_score_test_list.append(f1_score_test)
    
    classification_report_test_data = classification_report(test['Target'],test_pred)
    print("classification report ",classification_report_test_data)
    
    # class_x_un1 = tfidf_vect.transform(unlabel_1['text'])
    # pseudo_pred = B.predict(class_x_un1)

    class_x_un1 = tfidf_vect.transform(unlabel_1['text'])

    class_pred_unlabel_1 = B.predict(class_x_un1)
    unlabel_1['Target']=class_pred_unlabel_1
    frame_1 = [train,unlabel_1]
    train_1 = pd.concat(frame_1)
    
    print(len(train_1))
    
    
    
    with open(r'C:\Users\iia\Documents\Supriya\Co_training_results_3C\unlabel_test.txt', 'a') as writefile:
        writefile.write("\n")
        writefile.write(" Co-training results ")
        writefile.write("\n")
        writefile.write(" train_size ")
        writefile.write("\n")
        writefile.write(str(train_size))
        writefile.write(" Accuracy_score_rf = ")
        writefile.write(str(Accuracy_score_rf))
        writefile.write(" Accuracy_score_lgb = ")
        writefile.write(str(Accuracy_score_lgb))
        writefile.write("\n")
        writefile.write(" Accuracy_score_svm = ")
        writefile.write(str(Accuracy_score_svm))
        writefile.write("\n")
        writefile.write(" classification_report_test_rf = ")
        writefile.write("\n")
        writefile.write(str(classification_report_test_rf))
        writefile.write("\n")
        writefile.write(" classification_report_test_lgb = ")
        writefile.write("\n")
        writefile.write(str(classification_report_test_lgb))
        writefile.write("\n")
        writefile.write(" classification_report_test_svm = ")
        writefile.write("\n")
        writefile.write(str(classification_report_test_svm))
        writefile.write("\n")
      
        writefile.write(" cross_valid_rf = ")
        writefile.write(str(cross_valid_rf))
        writefile.write("\n")
        writefile.write(" cross_valid_lgb = ")
        writefile.write(str(cross_valid_lgb))
        writefile.write("\n")
        writefile.write(" cross_valid_svm = ")
        writefile.write(str(cross_valid_svm))
        writefile.write("\n")
        
        writefile.write(" f1_score_rf = ")
        writefile.write(str(f1_score_rf))
        writefile.write("\n")
        writefile.write(" f1_score_lgb = ")
        writefile.write(str(f1_score_lgb))
        writefile.write("\n")
        writefile.write(" f1_score_svm = ")
        writefile.write(str(f1_score_svm))
        writefile.write("\n")
        writefile.write(" Results on test data ")
        writefile.write("\n")
        writefile.write(" f1_score_test = ")
        writefile.write(str(f1_score_test))
        writefile.write("\n")
        writefile.write(" classifier selected = ")
        writefile.write(str(B))
        writefile.write("\n")
        writefile.write(" classification_report_test_data = ")
        writefile.write(str(classification_report_test_data))
        writefile.write("\n")
    
    
    
    return train_1,train_size_list,f1_score_test_list




    # print('Validation Accuracy_score: %f' % Accuracy_score)
    # test_pred = classifier.predict(t_p)
    # acc_test = accuracy_score(test_pred,test['Target'])
    # print('Accuracy test data', acc_test)
    # matrix = confusion_matrix(y_pred, y_val)
    # print(matrix)


In [ ]:
import pandas as pd

In [ ]:
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

In [ ]:
unlabel = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Unlabeled_data.csv')
unlabel.head()


del unlabel['Complete']

del unlabel['Unnamed: 0']

unlabel.head()

unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

unlabel.head()


len(unlabel)



# unlabel_1 = unlabel.loc[:100]
# unlabel_1.head()

In [ ]:
train1 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data1.csv')
train2 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data2.csv')
train3 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data3.csv')
train4 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data4.csv')
train5 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data5.csv')
train6 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data6.csv')
train7 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data7.csv')
train8 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data8.csv')
train9 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data9.csv')
train10 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data10.csv')
train_highKappa = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data_highkappa.csv')

train1.head()
train = train1
train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
for i in train_list:
    #print(i)
    train = train.append(i)

#unlabel_size = unlabel_size
#Threshold=Threshold
train.sort_values("Sentence", inplace = True)
print(len(train))

train = train.drop_duplicates(subset ="Sentence")

train['Target'].unique()


train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()
print(len(train))
train = train.rename(columns={'Sentence':'text'})
all_Data_train_size = len(train)

In [ ]:
length_list = range(1000,100000,5000)

In [ ]:
j=0
iteration = 0
i_list = []
j_list = []
f1_score_test_list = []
train_size_list =[]

for i in length_list:
  print("iteration ",iteration)
  print(i)
  print(j)
  unlabel_chunk = unlabel[j:i]
  train,op_train_size_list,op_f1_score_test_list = training(unlabel_chunk,train,i,j)
  j=i
  iteration = iteration + 1
  print("length",len(unlabel_chunk))
    
with open(r'C:\Users\iia\Documents\Supriya\Co_training_results_3C\unlabel_test_variables.txt', 'a') as writefile:
    writefile.write("\n")
    writefile.write(" Co-training results ")
    writefile.write("\n")
    writefile.write("\n")
    writefile.write(" train size ")
    writefile.write(str(all_Data_train_size))
    writefile.write("\n")
#     writefile.write("op_unlabel_size")
#     writefile.write("\n")
#     writefile.write(str(op_unlabel_size))
    writefile.write("\n")
    writefile.write("op_train_size_list")
    writefile.write("\n")
    writefile.write(str(op_train_size_list))
    writefile.write("\n")
    writefile.write("op_f1_score_test_list")
    writefile.write("\n")
    writefile.write(str(op_f1_score_test_list))
    


In [ ]:
with open(r'C:\Users\iia\Documents\Supriya\Co_training_results\unlabel_test_variables.txt', 'a') as writefile:
    writefile.write("\n")
    writefile.write(" Co-training results ")
    writefile.write("\n")
    writefile.write("\n")
    writefile.write(" train size ")
    writefile.write(str(all_Data_train_size))
    writefile.write("\n")
#     writefile.write("op_unlabel_size")
#     writefile.write("\n")
#     writefile.write(str(op_unlabel_size))
    writefile.write("\n")
    writefile.write("op_train_size_list")
    writefile.write("\n")
    writefile.write(str(op_train_size_list))
    writefile.write("\n")
    writefile.write("op_f1_score_test_list")
    writefile.write("\n")
    writefile.write(str(op_f1_score_test_list))
    